In [9]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import math
import random

In [10]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

# colors = np.random.randint(0, 255, (len(labels_index), 3))
# colors = np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')
# colors[labels_index['BackG']] = 1.

In [14]:
# Build training and testing data

def compute_intersection_kernel_oneJob(i, j1):
    dist = np.minimum(train_data_normalized[i], train_data_normalized[j1:]).sum(axis=1)
    return dist

sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'

exprmt = 10
spm_level = 0 #0 or 2
train_frac = 0.8
n_sample = 1000 #Per class

for e in xrange(exprmt):
    
    t = time.time()

    train_data = []
    train_labels = []
    train_fnames = []

    test_data = []
    test_labels = []
    test_fnames = []

    for name in labels:

        files = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_fnames.bp' % {'name': name})
        hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
        if spm_level==2:
            hists1 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l1.bp' % {'name': name})
            hists2 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l2.bp' % {'name': name})

        n_total = hists0.shape[0]
        print (name, n_total),

        random_indices = np.random.choice(range(n_total), min(n_total, n_sample), replace=False)

        frac = int(train_frac*len(random_indices))
        random_train = random_indices[:frac]
        random_test = random_indices[frac:]
        n_train = len(random_train)
        n_test = len(random_test)

        if spm_level==0:
            train_hists = np.c_[hists0[random_train]]
            test_hists =  np.c_[hists0[random_test]]
        elif spm_level==2:
            train_hists = np.c_[hists0[random_train], 
                            hists1[random_train].reshape((n_train, -1)), 
                            hists2[random_train].reshape((n_train, -1))]
            test_hists =  np.c_[hists0[random_test], 
                            hists1[random_test].reshape((n_test, -1)), 
                            hists2[random_test].reshape((n_test, -1))] 

        train_files = files[random_train].reshape((n_train,))
        test_files = files[random_test].reshape((n_test,))

        train_data.append(train_hists)
        train_labels.append(np.ones((n_train, )) * labels_index[name])
        train_fnames.append(train_files)

        test_data.append(test_hists)
        test_labels.append(np.ones((n_test, )) * labels_index[name])
        test_fnames.append(test_files)

        print 'Train: ', n_train, 
        print 'Test: ', n_test

    train_data = np.concatenate(train_data)
    train_labels = np.concatenate(train_labels)
    train_fnames = np.concatenate(train_fnames)
    n_train = train_data.shape[0]

    test_data = np.concatenate(test_data)
    test_labels = np.concatenate(test_labels)
    test_fnames = np.concatenate(test_fnames)
    n_test = test_data.shape[0]
    
    print train_data.shape
    print train_labels.shape
    print train_fnames.shape, train_fnames.dtype
    print train_fnames[0]

    print test_data.shape
    print test_labels.shape
    print test_fnames.shape, test_fnames.dtype
    print test_fnames[0]
    
    train_data_normalized = train_data / train_data.sum(axis=1)[:,None].astype(np.float)
    train_data_normalized = np.nan_to_num(train_data_normalized)

    test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)
    test_data_normalized = np.nan_to_num(test_data_normalized)
    
    train_dist_triangle = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, i) 
                                         for i in range(n_train)))

    train_dist_mat = np.empty((n_train, n_train))
    train_dist_mat[np.triu_indices(n_train)] = train_dist_triangle
    r = np.tril_indices(n_train)
    train_dist_mat[r] = train_dist_mat.T[r]
    print 'Train kernel shape: ', train_dist_mat.shape
    
    bp.pack_ndarray_file(train_data_normalized, sift_dir+'/preprocessed/train_level%d_t%d_e%d_data.bp'%(spm_level,int(train_frac*100),e))
    bp.pack_ndarray_file(train_labels, sift_dir+'/preprocessed/train_level%d_t%d_e%d_labels.bp'%(spm_level,int(train_frac*100),e))
    bp.pack_ndarray_file(train_fnames, sift_dir+'/preprocessed/train_level%d_t%d_e%d_fnames.bp'%(spm_level,int(train_frac*100),e), chunk_size=255*4000)

    bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/test_level%d_t%d_e%d_data.bp'%(spm_level,int(train_frac*100),e))
    bp.pack_ndarray_file(test_labels, sift_dir+'/preprocessed/test_level%d_t%d_e%d_labels.bp'%(spm_level,int(train_frac*100),e))
    bp.pack_ndarray_file(test_fnames, sift_dir+'/preprocessed/test_level%d_t%d_e%d_fnames.bp'%(spm_level,int(train_frac*100),e), chunk_size=255*4000)
    
    bp.pack_ndarray_file(train_dist_mat, sift_dir+'/preprocessed/train_level%d_t%d_e%d_spm_dist_mat.bp'%(spm_level,int(train_frac*100),e))
    

    sys.stderr.write('Experiment #%d Time: %f seconds\n' % (e, (time.time()-t)))

('BackG', 5105) Train:  800 Test:  200
('5N', 1440) Train:  800 Test:  200
('7n', 3444) Train:  800 Test:  200
('7N', 2579) Train:  800 Test:  200
('12N', 1230) Train:  800 Test:  200
('Pn', 3042) Train:  800 Test:  200
('VLL', 1287) Train:  800 Test:  200
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  800 Test:  200
('Tz', 1387) Train:  800 Test:  200
('RtTg', 2639) Train:  800 Test:  200
('LRt', 1050) Train:  800 Test:  200
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  800 Test:  200
(10769, 200)
(10769,)
(10769,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0247_0019.png
(2695, 200)
(2695,)
(2695,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0294_0001.png


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:90: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:93: RuntimeWarning: invalid value encountered in divide


Train kernel shape:  (10769, 10769)
('BackG', 5105) Train:  800 Test:  200
('5N', 1440) Train:  800 Test:  200
('7n', 3444) Train:  800 Test:  200
('7N', 2579) Train:  800 Test:  200
('12N', 1230) Train:  800 Test:  200
('Pn', 3042) Train:  800 Test:  200
('VLL', 1287) Train:  800 Test:  200
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  800 Test:  200
('Tz', 1387) Train:  800 Test:  200
('RtTg', 2639) Train:  800 Test:  200
('LRt', 1050) Train:  800 Test:  200
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  800 Test:  200
(10769, 200)
(10769,)
(10769,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0305_0016.png
(2695, 200)
(2695,)
(2695,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0150_0004.png


Experiment #0 Time: 15.556752 seconds


Train kernel shape:  (10769, 10769)
('BackG', 5105) Train:  800 Test:  200
('5N', 1440) Train:  800 Test:  200
('7n', 3444) Train:  800 Test:  200
('7N', 2579) Train:  800 Test:  200
('12N', 1230) Train:  800 Test:  200
('Pn', 3042) Train:  800 Test:  200
('VLL', 1287) Train:  800 Test:  200
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  800 Test:  200
('Tz', 1387) Train:  800 Test:  200
('RtTg', 2639) Train:  800 Test:  200
('LRt', 1050) Train:  800 Test:  200
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  800 Test:  200
(10769, 200)
(10769,)
(10769,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0174_0039.png
(2695, 200)
(2695,)
(2695,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0316_0012.png


...

Experiment #7 Time: 15.829519 seconds


Train kernel shape:  (10769, 10769)
('BackG', 5105) Train:  800 Test:  200
('5N', 1440) Train:  800 Test:  200
('7n', 3444) Train:  800 Test:  200
('7N', 2579) Train:  800 Test:  200
('12N', 1230) Train:  800 Test:  200
('Pn', 3042) Train:  800 Test:  200
('VLL', 1287) Train:  800 Test:  200
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  800 Test:  200
('Tz', 1387) Train:  800 Test:  200
('RtTg', 2639) Train:  800 Test:  200
('LRt', 1050) Train:  800 Test:  200
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  800 Test:  200
(10769, 200)
(10769,)
(10769,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0216_0006.png
(2695, 200)
(2695,)
(2695,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0187_0007.png


Experiment #8 Time: 16.321434 seconds


Train kernel shape:  (10769, 10769)


Experiment #9 Time: 15.564782 seconds
